In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_LAUNCH_BLOCKING=1
import torch

from main import init_model, init_training, create_connections_datasets

env: CUDA_LAUNCH_BLOCKING=1


/Users/brianfitzgerald/Documents/GitHub/minRL/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer, model = init_model()

2025-05-05 20:57:57.588 | INFO     | main:init_model:42 - Model loaded.
2025-05-05 20:57:57.590 | INFO     | main:init_model:45 - Using device mps


In [19]:
train_dataset, _ = create_connections_datasets(tokenizer)

In [4]:
training_objects = init_training(model, train_dataset)

In [20]:
from main import init_training
from grpo import rollout, update_policy

from tasks.countdown import reward_function

to = init_training(model, train_dataset)
batch = next(iter(to.train_dataloader))

episodes = rollout(
    model=model,
    tokenizer=tokenizer,
    batch=batch,
    max_new_tokens=to.config.max_new_tokens,
    num_answer_per_question=to.config.num_answer_per_question,
    reward_function=reward_function,
    device=to.device,
)
if to.config.skip_unfinished_episodes:
    episodes = [episode for episode in episodes if episode.is_finished]

2025-05-05 21:29:51.909 | INFO     | grpo:rollout:43 - Generating responses for 1 prompts, max_tokens=512
2025-05-05 21:30:15.314 | INFO     | grpo:rollout:73 - Generated token ids: 387
2025-05-05 21:30:15.315 | INFO     | grpo:rollout:80 - Generated token ids after removing padding: 387
2025-05-05 21:30:15.316 | INFO     | grpo:rollout:87 - Generated text: 

Okay, let's see. The user provided a list of words: copy, text, words, writing, agent, asset, mole, sleeper, choice, say, voice, vote, fiat, jaguar, mini. They want me to group them into four groups of four, each related to each other. Each word can only be in one group.

First, I need to look for obvious connections. Let's start with "copy, text, words, writing". These all seem related to writing. Copy can mean to copy text, text is part of writing, words are the elements of writing, and writing is the activity. So that's a group: copy, text, words, writing.

Next, "agent, asset, mole, sleeper". Agent is involved in an asset, ass

rewards {'reward': 0.010000000000000002, 'reward_info': {'format_reward': 0.1, 'answer_reward': 0.0}}


In [21]:
episodes[0].reward

0.010000000000000002

In [22]:
# Update policy - compute loss and perform backward pass
results = update_policy(
    model=model,
    optimizer=to.optimizer,
    episodes=episodes,
    micro_batch_size=to.config.micro_batch_size,
    pad_token_id=tokenizer.pad_token_id,
    max_grad_norm=to.config.max_grad_norm,
    device=to.device,
    dtype=to.dtype,
    apply_loss=True
)


2025-05-05 21:30:56.136 | INFO     | grpo:update_policy:180 - Updating policy with 1 episodes, 387 target tokens, 1 micro-batches
* Computing policy gradient:  0/ 1mINFO     | grpo:update_policy:187 - 


episodes [0.010000000000000002]
rewards tensor([0.0100], device='mps:0')
mean tensor(0.0100, device='mps:0')
std 0
batch ep [0.0]


In [24]:
results

{'loss': -0.0, 'grad_norm': 0.0, 'entropy': 1.5390625}